### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.45' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.45'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.45 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.5.12 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.45


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.45'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.45'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.45'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.45'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  321 Jun 27 18:14 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1824 Jun 27 18:14 dockerfile_aml-sdk_docker_image_sdk.v1.0.45


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.45 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.45 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 46500416382c
Removing intermediate container 46500416382c
 ---> df720df8bc26
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 4bc10e98a815
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:3 http://security.debian.org/debian-security stretch/updates InRelease [94.3 kB]
Get:4 http://deb.debian.org/debian stretch Release [118 kB]
Get:5 http://deb.debian.org/debian stretch Release.gpg [2434 B]
Get:6 http://deb.debian.org/debian stretch-updates/main amd64 Packages [27.2 kB]
Get:7 http://security.d

debconf: delaying package configuration, since apt-utils is not installed
Fetched 43.6 MB in 0s (98.1 MB/s)
Selecting previously unselected package libpython3.5-minimal:amd64.
(Reading database ... 10409 files and directories currently installed.)
Preparing to unpack .../00-libpython3.5-minimal_3.5.3-1+deb9u1_amd64.deb ...
Unpacking libpython3.5-minimal:amd64 (3.5.3-1+deb9u1) ...
Selecting previously unselected package python3.5-minimal.
Preparing to unpack .../01-python3.5-minimal_3.5.3-1+deb9u1_amd64.deb ...
Unpacking python3.5-minimal (3.5.3-1+deb9u1) ...
Selecting previously unselected package python3-minimal.
Preparing to unpack .../02-python3-minimal_3.5.3-1_amd64.deb ...
Unpacking python3-minimal (3.5.3-1) ...
Selecting previously unselected package mime-support.
Preparing to unpack .../03-mime-support_3.60_all.deb ...
Unpacking mime-support (3.60) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../04-libmpdec2_2.4.2-1_amd64.deb ...
Unpacking li

Selecting previously unselected package make.
Preparing to unpack .../36-make_4.1-9.1_amd64.deb ...
Unpacking make (4.1-9.1) ...
Selecting previously unselected package sudo.
Preparing to unpack .../37-sudo_1.8.19p1-2.1_amd64.deb ...
Unpacking sudo (1.8.19p1-2.1) ...
Setting up libquadmath0:amd64 (6.3.0-18+deb9u1) ...
Setting up libnpth0:amd64 (1.3-1) ...
Setting up libgomp1:amd64 (6.3.0-18+deb9u1) ...
Setting up libatomic1:amd64 (6.3.0-18+deb9u1) ...
Setting up readline-common (7.0-3) ...
Setting up libcc1-0:amd64 (6.3.0-18+deb9u1) ...
Setting up make (4.1-9.1) ...
Setting up libasan3:amd64 (6.3.0-18+deb9u1) ...
Setting up apt-transport-https (1.4.9) ...
Setting up mime-support (3.60) ...
Setting up sudo (1.8.19p1-2.1) ...
Setting up libcilkrts5:amd64 (6.3.0-18+deb9u1) ...
Setting up libreadline7:amd64 (7.0-3) ...
Setting up libubsan0:amd64 (6.3.0-18+deb9u1) ...
Setting up libtsan0:amd64 (6.3.0-18+deb9u1) ...
Setting up linux-libc-dev:amd64 (4.9.168-1+deb9u3) ...
Setting up distro-inf

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
mkl-2019.4-243.tar.bz2                     204.1 MB
nb_conda-2.2.1-py36_0.tar.bz2                 33 KB
joblib-0.13.2-py36_0.tar.bz2                 365 KB
parso-0.4.0-py_0.tar.bz2                      66 KB
wheel-0.33.4-py36_0.tar.bz2                   40 KB
mkl_fft-1.0.12-py36ha843d7b_0.tar.bz2        172 KB
setuptools-41.0.1-py36_0.tar.bz2             656 KB
wcwidth-0.1.7-py36_0.tar.bz2                  25 KB
ncurses-6.1-he6710b0_1.tar.bz2               958 KB
pygments-2.4.2-py_0.tar.bz2                  664 KB
pexpect-4.7.0-py36_0.tar.bz2                  82 KB
ca-certificates-2019.5.15-0.tar.bz2          133 KB
bleach-3.1.0-py36_0.tar.bz2                  226 KB
nb_conda_kernels-2.2.2-py36_0.tar.bz2         37 KB
ptyprocess-0.6.0-py36_0.tar.bz2               23 KB
numpy-1.16.4-py36h7e9f1db_0.tar.bz2           49 KB
tk-8.6.8-hbc83047_0.tar.bz2                  3.1 MB
pyzmq-18.0.0-py

 ---> Running in d3cab003e56a
Removing intermediate container d3cab003e56a
 ---> 63be2dd6aff4
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 8290a270b4ef
Removing intermediate container 8290a270b4ef
 ---> 57ee5e03b3a4
Successfully built 57ee5e03b3a4
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.45
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> df720df8bc26
Step 3/17 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 6601d8f38fb8
Step 4/17 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> Using cache
 ---> 4fc21ffd6462
Step 5/17 : ENV CONDA

  4900K .......... .......... .......... .......... .......... 78%  147M 0s
  4950K .......... .......... .......... .......... .......... 79%  164M 0s
  5000K .......... .......... .......... .......... .......... 80%  133M 0s
  5050K .......... .......... .......... .......... .......... 80%  238M 0s
  5100K .......... .......... .......... .......... .......... 81%  253M 0s
  5150K .......... .......... .......... .......... .......... 82%  208M 0s
  5200K .......... .......... .......... .......... .......... 83%  134M 0s
  5250K .......... .......... .......... .......... .......... 84%  177M 0s
  5300K .......... .......... .......... .......... .......... 84%  245M 0s
  5350K .......... .......... .......... .......... .......... 85%  241M 0s
  5400K .......... .......... .......... .......... .......... 86%  299M 0s
  5450K .......... .......... .......... .......... .......... 87%  152M 0s
  5500K .......... .......... .......... .......... .......... 88%  127M 0s
  5550K ....

In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 8
-rwxrwxrwx 1 root root 5031 Jun 24 15:29 exported_conda_env.yml


In [11]:
# export the resulted conda env
!docker run  \
        -v $crt_local_dir:/workspace:rw \
        $aml_docker_image_name \
        /bin/bash -c "conda env export > /workspace/exported_conda_env.yml"

In [12]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [13]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.45 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [14]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.45 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.45
1.0.45
azure-cli                         2.0.67

acr                                2.2.9
acs                                2.4.4
advisor                            2.0.1
ams                                0.4.7
appservice                        0.2.21
backup                             1.2.5
batch                              4.0.3
batchai                           0.4.10
billing                            0.2.2
botservice                         0.2.2
cdn                                0.2.4
cloud                              2.1.1
cognitiveservices                  0.2.6
command-modules-nspkg               2.0.2
configure                         2.0.24
consumption                        0.4.4
container                         0.3.18
core                              2.0.67
cosmosdb                          0.2.11
deploymentmanager          

#### Do the same in a  notebook

In [15]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [16]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [17]:
!ls -l $crt_local_dir

total 12
-rwxrwxrwx 1 root       root       5030 Jun 27 18:19 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001  982 Jun 27 18:19 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [18]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [19]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.45\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [20]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

4d5b75f3: Preparing 
fffd6909: Preparing 
a575c5aa: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 


4d5b75f3: Pushing  1.907GB/2.538GB

4d5b75f3: Pushed   2.591GB/2.538GBPushing  2.143GB/2.538GBsdk.v1.0.45: digest: sha256:55da8e95dac2716a0a20b18b942d332c44a82cc1af34c9850ba71d02fb59c753 size: 2003
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

2a5f464b: Preparing 
2ba78f2a: Preparing 
2dd221b1: Preparing 
4d5b75f3: Preparing 
fffd6909: Preparing 
a575c5aa: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
2dd221b1: Pushed   458.4MB/423.5MBraccount/aml-sdk_docker_image sdk.v1.0.45: digest: sha256:39140da9832cd37b6cb0dbac640af7d5731416a7a482319fee612c92e297e9aa size: 2634
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

f9763d87: Preparing 
2a5f464b: Preparing 
2ba78f2a: Preparing 
2dd221b1: Preparing 
4d5b75f3: Preparing 
fffd6909: Preparing 
a575c5aa: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
9763d87: Pushed   16.98MBists MBeraccount/a

In [21]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 500063 bytes to createAMLSDKDocker.html
